In [76]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import pandas as pd
import numpy as np
import xgboost as xgb
import sklearn.model_selection as skms
import warnings
warnings.filterwarnings('ignore')
import sklearn.linear_model as sklm
import sklearn.metrics as skm
import sklearn.base as skb

In [77]:
train_data = pd.read_csv(r'D:\Pycharm\project\Predict-the-cost-to-ship-the-sculptures\input\train_processed.csv')
test_data = pd.read_csv(r'D:\Pycharm\project\Predict-the-cost-to-ship-the-sculptures\input\test_processed.csv')
test_old=pd.read_csv(r'D:\Pycharm\project\Predict-the-cost-to-ship-the-sculptures\input\test.csv')

In [78]:
train_data.head()

,Artist Reputation,Height,Width,Weight,Price Of Sculpture,Base Shipping Price,Day,Material_Brass,Material_Bronze,Material_Clay,...,Express Shipment_Yes,Installation Included_Yes,Transport_Roadways,Transport_Unknown,Transport_Waterways,Fragile_Yes,Customer Information_Working Class,Remote Location_Unknown,Remote Location_Yes,Cost
0,0.26,17,6,4128,13.91,16.27,4,1,0,0,...,1,0,0,0,0,0,1,0,0,283.29
1,0.28,3,3,61,6.83,15.00,1,1,0,0,...,0,0,1,0,0,0,1,0,0,159.96
2,0.07,8,5,237,4.96,21.18,1,0,0,1,...,0,0,1,0,0,1,1,0,1,154.29
3,0.12,9,0,0,5.81,16.31,4,0,0,0,...,0,0,0,1,0,0,0,0,1,161.16
4,0.15,17,6,324,3.18,11.94,4,0,0,0,...,1,1,0,0,0,0,1,0,0,159.23


In [79]:
def log1p(vec):
    return np.log1p(abs(vec))


# apply log on target feature with abs as it has many negative values
train_data['Cost'] = log1p(train_data['Cost'])
train_data['Cost'].min()

4.466712392840524

In [80]:
seed =12

# shuffle samples
df_shuffle = train_data.sample(frac=1, random_state=seed).reset_index(drop=True)

df_y = df_shuffle.pop('Cost')
df_X = df_shuffle

# split into train dev and test
X_train, X_test, y_train, y_test = skms.train_test_split(df_X, df_y, train_size=0.8, random_state=seed)
print(f"Train set has {X_train.shape[0]} records out of {len(df_shuffle)} which is {round(X_train.shape[0]/len(df_shuffle)*100)}%")
print(f"Test set has {X_test.shape[0]} records out of {len(df_shuffle)} which is {round(X_test.shape[0]/len(df_shuffle)*100)}%")




Train set has 5200 records out of 6500 which is 80%
Test set has 1300 records out of 6500 which is 20%


In [81]:
import sklearn.preprocessing as skp

def getRmse(y_train, y_train_pred):
    print(skm.mean_squared_error(y_train, y_train_pred))
    
# scaler = skp.RobustScaler()
scaler = skp.MinMaxScaler()
# scaler = skp.StandardScaler()

# apply scaling to all numerical variables except dummy variables as they are already between 0 and 1
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

# scale test data with transform()
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)

# view sample data
X_train.describe()

,Artist Reputation,Height,Width,Weight,Price Of Sculpture,Base Shipping Price,Day,Material_Brass,Material_Bronze,Material_Clay,...,International_Yes,Express Shipment_Yes,Installation Included_Yes,Transport_Roadways,Transport_Unknown,Transport_Waterways,Fragile_Yes,Customer Information_Working Class,Remote Location_Unknown,Remote Location_Yes
count,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,...,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000
mean,0.411688,0.280854,0.193765,0.002999,0.003200,0.303077,0.505192,0.130577,0.123654,0.123077,...,0.334038,0.331346,0.404038,0.318077,0.208654,0.188654,0.158462,0.738077,0.115385,0.173654
std,0.292476,0.173775,0.129125,0.021129,0.024176,0.297448,0.303935,0.336969,0.329218,0.328557,...,0.471699,0.470742,0.490752,0.465774,0.406386,0.391271,0.365208,0.439723,0.319516,0.378848
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.160000,0.150685,0.111111,0.000003,0.000006,0.074461,0.250000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.390000,0.260274,0.177778,0.000019,0.000013,0.148533,0.500000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,0.650000,0.397260,0.244444,0.000224,0.000218,0.530007,0.750000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [82]:
model_xgb = xgb.XGBRegressor()

model_xgb.fit(X_train, y_train, early_stopping_rounds=7, eval_set=[(X_test, y_test)], verbose=1)

[0]	validation_0-rmse:4.43335
[1]	validation_0-rmse:3.12819
[2]	validation_0-rmse:2.21708
[3]	validation_0-rmse:1.57841
[4]	validation_0-rmse:1.14234
[5]	validation_0-rmse:0.84838
[6]	validation_0-rmse:0.64596
[7]	validation_0-rmse:0.52125
[8]	validation_0-rmse:0.44216
[9]	validation_0-rmse:0.39090
[10]	validation_0-rmse:0.36249
[11]	validation_0-rmse:0.34473
[12]	validation_0-rmse:0.33532
[13]	validation_0-rmse:0.33106
[14]	validation_0-rmse:0.32684
[15]	validation_0-rmse:0.32303
[16]	validation_0-rmse:0.32187
[17]	validation_0-rmse:0.31865
[18]	validation_0-rmse:0.31524
[19]	validation_0-rmse:0.31352
[20]	validation_0-rmse:0.31306
[21]	validation_0-rmse:0.31272
[22]	validation_0-rmse:0.31147
[23]	validation_0-rmse:0.30849
[24]	validation_0-rmse:0.30880
[25]	validation_0-rmse:0.30832
[26]	validation_0-rmse:0.30766
[27]	validation_0-rmse:0.30541
[28]	validation_0-rmse:0.30395
[29]	validation_0-rmse:0.30429
[30]	validation_0-rmse:0.30367
[31]	validation_0-rmse:0.30272
[32]	validation_0-

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [83]:
ridge_model = sklm.RidgeCV(scoring = "neg_mean_squared_error", 
                           alphas = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1, 1.0, 10], 
                           cv=5)
ridge_model.fit(X_train, y_train)

# predict
y_train_pred = ridge_model.predict(X_train)
y_test_pred = ridge_model.predict(X_test)
getRmse(y_train, y_train_pred)
getRmse(y_test, y_test_pred)

0.7086567142943605
0.6996688209931315


In [84]:
#Generate Ensembles
def rmse_cv(model):
    rmse = np.sqrt(-skms.cross_val_score(model, X_train, y_train, 
                                         scoring="neg_mean_squared_error", cv = 5, n_jobs=-1))
    return rmse

class MixModel(skb.BaseEstimator, skb.RegressorMixin, skb.TransformerMixin):

    def __init__(self, algs):
        self.algs = algs

    # Define clones of parameters models
    def fit(self, X, y):
        self.algs_ = [skb.clone(x) for x in self.algs]
        
        # Train cloned base models
        for alg in self.algs_:
            alg.fit(X, y)

        return self
    
    # Average predictions of all cloned models
    def predict(self, X):
        predictions = np.column_stack([
            stacked_model.predict(X) for stacked_model in self.algs_
        ])
        return np.mean(predictions, axis=1) 
    

In [85]:
mixed_model = MixModel(algs = [
   model_xgb,
    ridge_model
])
# score = rmse_cv(mixed_model)
# print("\nAveraged base algs score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

mixed_model.fit(X_train, y_train)

# predict
y_train_pred = mixed_model.predict(X_train)
y_test_pred = mixed_model.predict(X_test)
getRmse(y_train, y_train_pred)
getRmse(y_test, y_test_pred)

0.18387361422200488
0.23795848047158985


In [86]:
def clipExp(vec):
    return np.clip(expm1(vec), 0, None)
def expm1(x):
    return np.expm1(x)

def getTestResults(model=None, roundOff=False):
    targetFeature='Cost'
    df_final = train_data.sample(frac=1, random_state=1).reset_index(drop=True)
    test_cols = [x for x in df_final.columns if targetFeature not in x]
    df_test=test_data.copy()
    df_final_test = df_test
    df_y = df_final.pop(targetFeature)
    df_X = df_final

#     scaler = skp.RobustScaler()
    scaler = skp.MinMaxScaler()
#     scaler = skp.StandardScaler()

    df_X = pd.DataFrame(scaler.fit_transform(df_X), columns=df_X.columns)
    X_test = pd.DataFrame(scaler.transform(df_final_test), columns=df_X.columns)
    
    # for machine learning models, else neural network will work
    if model is None:
        model = MixModel(algs = [
#             ridge_model, 
#             enet_model, 
            model_xgb,
            ridge_model
        ])

        model.fit(df_X, df_y)

    # predict
    y_train_pred = model.predict(df_X)
    y_test_pred = model.predict(X_test)
    
    if roundOff:
        y_train_pred = np.round(y_train_pred)
        y_test_pred = np.round(y_test_pred)
                
    if type(y_test_pred[0]) == np.ndarray:
        y_test_pred = np.ravel(y_test_pred)

    getRmse(df_y, y_train_pred)
    return clipExp(y_test_pred)

In [87]:
# ML models
results = getTestResults(roundOff=False)


0.18442608392166018


In [88]:
submission = pd.DataFrame({
    'Customer Id': test_old['Customer Id'],
    'Cost': results,
})
submission.head()

submission.to_csv('Sub1.csv',index=False)